In [1]:
import numpy as np
import pandas as pd

In [2]:
path_train_data = '/Users/denis/Desktop/ML/Kaggle/optiver-trading-at-the-close/'

In [3]:
%%time

train_df = pd.read_csv(path_train_data + 'train.csv')
print(f"Shape df_train is: {train_df.shape}")

Shape df_train is: (5237980, 17)
CPU times: user 3.71 s, sys: 949 ms, total: 4.66 s
Wall time: 5.12 s


In [8]:
def b_i(row):
    if row['imbalance_buy_sell_flag'] == 0:
        buy_interest = row['matched_size']
    elif row['imbalance_buy_sell_flag'] > 0:
        buy_interest = row['matched_size'] + row['imbalance_size']
    else:
        buy_interest = row['matched_size']
    return buy_interest

def s_i(row):
    if row['imbalance_buy_sell_flag'] == 0:
        sell_interest = row['matched_size']
    elif row['imbalance_buy_sell_flag'] > 0:
        sell_interest = row['matched_size']
    else:
        sell_interest = row['matched_size'] + row['imbalance_size']
    return sell_interest

In [9]:
%%time
train_df['buy_interest'] = train_df.apply(b_i, axis = 1)
train_df['sell_interest'] = train_df.apply(s_i, axis = 1)

CPU times: user 1min 9s, sys: 4.52 s, total: 1min 13s
Wall time: 1min 15s


In [14]:
train_df[(train_df['seconds_in_bucket'] == 540) & (train_df['stock_id'] == 0)].head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,buy_interest,sell_interest
10314,0,0,540,906957.07,1,0.999383,27183793.68,1.000241,0.999919,0.999276,111912.00,0.999383,9793.35,0.999375,9.759665,54,0_540_0,28090750.75,27183793.68
20819,0,1,540,875760.77,1,0.999816,13073132.71,1.001105,1.000353,0.999493,280239.13,0.999816,85596.80,0.999740,-2.700090,109,1_540_0,13948893.48,13073132.71
31324,0,2,540,0.00,0,0.998557,16122712.09,0.998557,0.998557,0.998557,9480.12,0.998774,18596.12,0.998631,2.019405,164,2_540_0,16122712.09,16122712.09
41829,0,3,540,4785325.00,-1,0.998280,22303737.18,0.990578,0.995243,0.998280,52917.25,0.998388,160333.68,0.998307,-8.149743,219,3_540_0,22303737.18,27089062.18
52388,0,4,540,2351751.43,-1,1.000129,29143550.50,0.996317,0.996935,1.000129,141901.72,1.000232,373331.22,1.000157,-12.429953,274,4_540_0,29143550.50,31495301.93


In [19]:
means = train_df.groupby(['date_id', 'stock_id'])['matched_size'].mean()

In [20]:
means.head(190)

date_id  stock_id
0        0           2.064913e+07
         1           2.535047e+06
         2           3.304662e+06
         3           3.102247e+07
         4           2.902312e+07
                         ...     
         193         7.904142e+07
         194         2.906223e+06
         195         2.087869e+07
         196         6.548406e+06
         197         5.987728e+06
Name: matched_size, Length: 190, dtype: float64

In [59]:
def generate_features(df, train_flag):
    
    df_out = df.drop(['far_price', 'near_price', 'row_id'], axis = 1)
    
    df_out['buy_interest'] = df_out.apply(b_i, axis = 1)
    df_out['sell_interest'] = df_out.apply(s_i, axis = 1)

    features = ['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_buy_sell_flag', 'imbalance_size', 'matched_size', 
                'bid_size', 'ask_size', 'reference_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2', 'imb_s3', 'buy_interest', 'sell_interest'
               ]
    
    df_out['imb_s1'] = df_out.eval('(ask_size - bid_size)/(ask_size + bid_size)')
    df_out['imb_s2'] = df_out.eval('(imbalance_size - matched_size)/(matched_size + imbalance_size)')
    df_out['imb_s3'] = np.log(df_out.ask_size) - np.log(df_out.bid_size)
    
    prices = ['reference_price', 'ask_price', 'bid_price', 'wap']
    
    for i, a in enumerate(prices):
        for j, b in enumerate(prices):
            if i > j:
                df_out[f'{a}_{b}_imb'] = df_out.eval(f'({a} - {b})/({a} + {b})')
                features.append(f'{a}_{b}_imb')    
                    
    for i, a in enumerate(prices):
        for j, b in enumerate(prices):
            for k, c in enumerate(prices):
                if i > j and j > k:
                    max_ = df_out[[a,b,c]].max(axis = 1)
                    min_ = df_out[[a,b,c]].min(axis = 1)
                    mid_ = df_out[[a,b,c]].sum(axis = 1) - min_ - max_

                    df_out[f'{a}_{b}_{c}_imb2'] = (max_ - mid_)/(mid_ - min_)
                    features.append(f'{a}_{b}_{c}_imb2')
    
    if train_flag:
        df_out = df_out.dropna()
        df_out.replace([np.inf, -np.inf], 0, inplace = True)
    else:
        df_out.fillna(0, inplace = True)
        df_out.replace([np.inf, -np.inf], 0, inplace = True)
        
    if train_flag:
        return df_out[features], df_out['target']
    else:
        return df_out[features], None

In [39]:
%%time

X, y = generate_features(train_df, True)

/opt/homebrew/Caskroom/miniforge/base/envs/boost/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/8z/rmv13nvj523fxpvqhcn4k7dm0000gn/T/ipykernel_71144/2811323902.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.replace([np.inf, -np.inf], 0, inplace = True)


CPU times: user 2.69 s, sys: 2.58 s, total: 5.27 s
Wall time: 6.78 s


In [40]:
X.isna().sum()

stock_id                                    0
date_id                                     0
seconds_in_bucket                           0
imbalance_buy_sell_flag                     0
imbalance_size                              0
matched_size                                0
bid_size                                    0
ask_size                                    0
reference_price                             0
ask_price                                   0
bid_price                                   0
wap                                         0
imb_s1                                      0
imb_s2                                      0
imb_s3                                      0
buy_interest                                0
sell_interest                               0
ask_price_reference_price_imb               0
bid_price_reference_price_imb               0
bid_price_ask_price_imb                     0
wap_reference_price_imb                     0
wap_ask_price_imb                 

In [41]:
y.isna().sum()

0

In [42]:
np.isinf(X).values.sum()

0

In [43]:
np.isinf(y).values.sum()

0

In [44]:
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

xgb.set_config(verbosity = 0)

In [45]:
def objective(trial, X = X, y = y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 1977)
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 2000, 8000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log = True),
        'max_depth': trial.suggest_int('max_depth', 3, 32),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'gamma': trial.suggest_float('gamma', 0.0001, 1.0, log = True),
        'alpha': trial.suggest_float('alpha', 0.0001, 10.0, log = True),
        'lambda': trial.suggest_float('lambda', 0.0001, 10.0, log = True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.8),
        'subsample': trial.suggest_float('subsample', 0.1, 0.8),
        "n_iter_no_change": 50,
    }
    
    dtrain = xgb.DMatrix(data = X_train, label = y_train)
    dval = xgb.DMatrix(data = X_val, label = y_val)

    regressor = xgb.train(params, dtrain) 
    
    y_pred = regressor.predict(dval)
    mae = mean_absolute_error(y_val, y_pred)

    return mae

In [46]:
%%time

# make a study
study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 200)

[I 2023-12-16 22:52:39,707] A new study created in memory with name: no-name-14cc08e7-799d-4686-bfd7-76253958dd5c
[I 2023-12-16 22:52:49,323] Trial 0 finished with value: 6.400495820840649 and parameters: {'n_estimators': 7802, 'learning_rate': 0.018089808504549037, 'max_depth': 5, 'min_child_weight': 166, 'gamma': 0.0008850537737845741, 'alpha': 4.745917101919948, 'lambda': 0.03954072454362793, 'colsample_bytree': 0.1710102705579013, 'subsample': 0.5651230892187913}. Best is trial 0 with value: 6.400495820840649.
[I 2023-12-16 22:53:20,907] Trial 1 finished with value: 6.420024236043219 and parameters: {'n_estimators': 2958, 'learning_rate': 0.0010779499286356152, 'max_depth': 9, 'min_child_weight': 128, 'gamma': 0.05172700915839884, 'alpha': 0.493417118410193, 'lambda': 0.00015846024642852973, 'colsample_bytree': 0.6248119733439962, 'subsample': 0.7850584349700837}. Best is trial 0 with value: 6.400495820840649.
[I 2023-12-16 22:54:10,257] Trial 2 finished with value: 6.3861919108192

[I 2023-12-16 23:14:20,703] Trial 20 finished with value: 6.2857393082485835 and parameters: {'n_estimators': 5429, 'learning_rate': 0.12676526136812472, 'max_depth': 23, 'min_child_weight': 263, 'gamma': 0.007133852160955239, 'alpha': 0.0007817848160169751, 'lambda': 0.22360874504450257, 'colsample_bytree': 0.3720115096527869, 'subsample': 0.6453108892353824}. Best is trial 17 with value: 6.280216929508274.
[I 2023-12-16 23:15:05,031] Trial 21 finished with value: 6.285990910318321 and parameters: {'n_estimators': 5593, 'learning_rate': 0.12136371242032089, 'max_depth': 22, 'min_child_weight': 269, 'gamma': 0.0056165252789714066, 'alpha': 0.0009150423141635218, 'lambda': 0.24111220472606337, 'colsample_bytree': 0.3618350628795947, 'subsample': 0.6499729033530967}. Best is trial 17 with value: 6.280216929508274.
[I 2023-12-16 23:15:53,719] Trial 22 finished with value: 6.277466677955566 and parameters: {'n_estimators': 6889, 'learning_rate': 0.2920113475574488, 'max_depth': 28, 'min_ch

[I 2023-12-16 23:30:17,853] Trial 40 finished with value: 6.37093209369041 and parameters: {'n_estimators': 5221, 'learning_rate': 0.02552880770947309, 'max_depth': 30, 'min_child_weight': 36, 'gamma': 0.2276215796942038, 'alpha': 0.07468032367707987, 'lambda': 0.0004948686372702324, 'colsample_bytree': 0.1581299732171704, 'subsample': 0.7701519028184177}. Best is trial 38 with value: 6.261470865224351.
[I 2023-12-16 23:31:12,552] Trial 41 finished with value: 6.301665511977697 and parameters: {'n_estimators': 5916, 'learning_rate': 0.06839791461849971, 'max_depth': 32, 'min_child_weight': 34, 'gamma': 0.13752610728266754, 'alpha': 0.0002289516939187796, 'lambda': 0.01291520481837636, 'colsample_bytree': 0.21990972132144715, 'subsample': 0.7945633270425843}. Best is trial 38 with value: 6.261470865224351.
[I 2023-12-16 23:32:00,464] Trial 42 finished with value: 6.2717645692692985 and parameters: {'n_estimators': 5778, 'learning_rate': 0.18760523650804714, 'max_depth': 28, 'min_child_w

[I 2023-12-16 23:47:11,439] Trial 60 finished with value: 6.296418598433668 and parameters: {'n_estimators': 6951, 'learning_rate': 0.12277469931342878, 'max_depth': 31, 'min_child_weight': 282, 'gamma': 0.06493078279938166, 'alpha': 0.2878265066170856, 'lambda': 0.0008199215642086736, 'colsample_bytree': 0.20502365148901142, 'subsample': 0.720186236089608}. Best is trial 38 with value: 6.261470865224351.
[I 2023-12-16 23:47:53,190] Trial 61 finished with value: 6.283050474444535 and parameters: {'n_estimators': 7459, 'learning_rate': 0.14819528814085914, 'max_depth': 24, 'min_child_weight': 69, 'gamma': 0.11886752914740535, 'alpha': 1.4868548302136455, 'lambda': 0.004360487387262618, 'colsample_bytree': 0.23345096801391918, 'subsample': 0.7981992553389452}. Best is trial 38 with value: 6.261470865224351.
[I 2023-12-16 23:48:39,670] Trial 62 finished with value: 6.278440622762725 and parameters: {'n_estimators': 7728, 'learning_rate': 0.16105102850648653, 'max_depth': 25, 'min_child_we

[I 2023-12-17 00:04:19,139] Trial 80 finished with value: 6.291664410963717 and parameters: {'n_estimators': 6047, 'learning_rate': 0.22529399935373198, 'max_depth': 30, 'min_child_weight': 264, 'gamma': 0.4680951615099119, 'alpha': 0.00026077021025359027, 'lambda': 0.0051897978646583505, 'colsample_bytree': 0.37902240813518906, 'subsample': 0.3983419317032979}. Best is trial 38 with value: 6.261470865224351.
[I 2023-12-17 00:05:54,178] Trial 81 finished with value: 6.27405049792963 and parameters: {'n_estimators': 6302, 'learning_rate': 0.14903747898389696, 'max_depth': 31, 'min_child_weight': 277, 'gamma': 0.3234376225768068, 'alpha': 0.0004165709046684534, 'lambda': 0.00844938944867756, 'colsample_bytree': 0.5108781429296827, 'subsample': 0.7633317257126605}. Best is trial 38 with value: 6.261470865224351.
[I 2023-12-17 00:07:18,436] Trial 82 finished with value: 6.27447592444184 and parameters: {'n_estimators': 6526, 'learning_rate': 0.17505115206932395, 'max_depth': 32, 'min_child

...

[I 2023-12-17 00:51:58,154] Trial 120 finished with value: 6.263887344458822 and parameters: {'n_estimators': 4892, 'learning_rate': 0.1879634590264275, 'max_depth': 32, 'min_child_weight': 238, 'gamma': 0.4468209574036797, 'alpha': 0.3957157195232608, 'lambda': 0.02576409794862753, 'colsample_bytree': 0.7763050010730481, 'subsample': 0.7155497874258433}. Best is trial 38 with value: 6.261470865224351.
[I 2023-12-17 00:54:00,353] Trial 121 finished with value: 6.260379937094507 and parameters: {'n_estimators': 4924, 'learning_rate': 0.19201291611033014, 'max_depth': 32, 'min_child_weight': 241, 'gamma': 0.418580010014932, 'alpha': 0.25939985901195967, 'lambda': 0.02468548982687798, 'colsample_bytree': 0.7797616879096914, 'subsample': 0.7151517168427026}. Best is trial 121 with value: 6.260379937094507.
[I 2023-12-17 00:55:59,226] Trial 122 finished with value: 6.264834398363426 and parameters: {'n_estimators': 4369, 'learning_rate': 0.18398091726110655, 'max_depth': 32, 'min_child_weig

[I 2023-12-17 01:39:41,330] Trial 140 finished with value: 6.301925212694717 and parameters: {'n_estimators': 4620, 'learning_rate': 0.06465089098763897, 'max_depth': 31, 'min_child_weight': 217, 'gamma': 0.0002518707847947685, 'alpha': 1.008866338331737, 'lambda': 0.2826287269114152, 'colsample_bytree': 0.7518265406194896, 'subsample': 0.7196728156840715}. Best is trial 121 with value: 6.260379937094507.
[I 2023-12-17 01:41:47,513] Trial 141 finished with value: 6.259649575708548 and parameters: {'n_estimators': 4505, 'learning_rate': 0.1645456905190602, 'max_depth': 32, 'min_child_weight': 233, 'gamma': 0.0002386018906165099, 'alpha': 0.5775567835357546, 'lambda': 0.11079330739400055, 'colsample_bytree': 0.7867304243590267, 'subsample': 0.729593809656841}. Best is trial 141 with value: 6.259649575708548.
[I 2023-12-17 01:43:51,578] Trial 142 finished with value: 6.260330985630019 and parameters: {'n_estimators': 4770, 'learning_rate': 0.15783861296210602, 'max_depth': 31, 'min_child_

[I 2023-12-17 02:21:18,099] Trial 160 finished with value: 6.27692377735847 and parameters: {'n_estimators': 4293, 'learning_rate': 0.12268724716314099, 'max_depth': 17, 'min_child_weight': 204, 'gamma': 0.0005183844937381249, 'alpha': 0.6347723566791154, 'lambda': 0.093928943900978, 'colsample_bytree': 0.760845689011006, 'subsample': 0.6798496054051776}. Best is trial 141 with value: 6.259649575708548.
[I 2023-12-17 02:23:31,116] Trial 161 finished with value: 6.262992528742163 and parameters: {'n_estimators': 4465, 'learning_rate': 0.15927553379438542, 'max_depth': 32, 'min_child_weight': 244, 'gamma': 0.000678679003671631, 'alpha': 1.5485120477372927, 'lambda': 0.07369628752814728, 'colsample_bytree': 0.7720853484568995, 'subsample': 0.7291214815659507}. Best is trial 141 with value: 6.259649575708548.
[I 2023-12-17 02:25:38,566] Trial 162 finished with value: 6.262838700640377 and parameters: {'n_estimators': 4499, 'learning_rate': 0.16886250979681242, 'max_depth': 31, 'min_child_w

[I 2023-12-17 03:00:53,026] Trial 180 finished with value: 6.268459279268685 and parameters: {'n_estimators': 4000, 'learning_rate': 0.25603365876353257, 'max_depth': 32, 'min_child_weight': 176, 'gamma': 0.00043050091958209347, 'alpha': 7.37872213452547, 'lambda': 0.48752011227934644, 'colsample_bytree': 0.7768920964851643, 'subsample': 0.7490540606508151}. Best is trial 179 with value: 6.258984601781205.
[I 2023-12-17 03:02:58,095] Trial 181 finished with value: 6.258930402863156 and parameters: {'n_estimators': 4267, 'learning_rate': 0.16987645395358383, 'max_depth': 30, 'min_child_weight': 165, 'gamma': 0.0008669890550996714, 'alpha': 1.3874741641288368, 'lambda': 0.06464543052706935, 'colsample_bytree': 0.781499920579469, 'subsample': 0.7322463216704231}. Best is trial 181 with value: 6.258930402863156.
[I 2023-12-17 03:05:03,752] Trial 182 finished with value: 6.26046621015363 and parameters: {'n_estimators': 4257, 'learning_rate': 0.19832709010091346, 'max_depth': 30, 'min_child

CPU times: user 1d 11h 42min 21s, sys: 1h 25min 23s, total: 1d 13h 7min 44s
Wall time: 4h 45min 21s


In [47]:
# Print the result
best_params = study.best_params
best_score = study.best_value
print(f"Best score: {best_score}\n")
print(f"Optimized parameters: {best_params}\n")

Best score: 6.258930402863156

Optimized parameters: {'n_estimators': 4267, 'learning_rate': 0.16987645395358383, 'max_depth': 30, 'min_child_weight': 165, 'gamma': 0.0008669890550996714, 'alpha': 1.3874741641288368, 'lambda': 0.06464543052706935, 'colsample_bytree': 0.781499920579469, 'subsample': 0.7322463216704231}



In [48]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.4, random_state = 1977)
dtrain = xgb.DMatrix(data = X_train, label = y_train)
dval = xgb.DMatrix(data = X_val, label = y_val)

regressor = xgb.train(best_params, dtrain)
y_pred = regressor.predict(dval)
mae = mean_absolute_error(y_val, y_pred)

print("Mean Absolute Error: {}".format(mae))

Mean Absolute Error: 6.274675763741373


In [49]:
%%time

path_test_data = '/Users/denis/Desktop/ML/Kaggle/optiver-trading-at-the-close/example_test_files/'

test_df = pd.read_csv(path_test_data + 'test.csv')
print(f"Shape df_train is: {test_df.shape}")

Shape df_train is: (33000, 16)
CPU times: user 35.7 ms, sys: 14 ms, total: 49.7 ms
Wall time: 65.6 ms


In [50]:
test_df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id
0,0,478,0,3753451.43,-1,0.999875,11548975.43,NaN,NaN,0.999875,22940.00,1.000050,9177.60,1.0,26290,478_0_0
1,1,478,0,985977.11,-1,1.000245,3850033.97,NaN,NaN,0.999940,1967.90,1.000601,19692.00,1.0,26290,478_0_1
2,2,478,0,599128.74,1,1.000584,4359198.25,NaN,NaN,0.999918,4488.22,1.000636,34955.12,1.0,26290,478_0_2
3,3,478,0,2872317.54,-1,0.999802,27129551.64,NaN,NaN,0.999705,16082.04,1.000189,10314.00,1.0,26290,478_0_3
4,4,478,0,740059.14,-1,0.999886,8880890.78,NaN,NaN,0.999720,19012.35,1.000107,7245.60,1.0,26290,478_0_4


In [51]:
test_targets = pd.read_csv(path_test_data + 'revealed_targets.csv')
test_targets.head()

,stock_id,date_id,seconds_in_bucket,time_id,revealed_target,revealed_date_id,revealed_time_id
0,0.0,478,0,26290,-2.310276,477.0,26235.0
1,1.0,478,0,26290,-12.850165,477.0,26235.0
2,2.0,478,0,26290,-0.439882,477.0,26235.0
3,3.0,478,0,26290,7.259846,477.0,26235.0
4,4.0,478,0,26290,4.780292,477.0,26235.0


In [52]:
df = pd.merge(test_df, test_targets[['date_id', 'seconds_in_bucket', 'stock_id', 'revealed_target']], how = 'outer', on = ['date_id', 'seconds_in_bucket', 'stock_id'])

In [53]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,revealed_target
0,0.0,478,0,3753451.43,-1.0,0.999875,11548975.43,NaN,NaN,0.999875,22940.00,1.000050,9177.60,1.0,26290.0,478_0_0,-2.310276
1,1.0,478,0,985977.11,-1.0,1.000245,3850033.97,NaN,NaN,0.999940,1967.90,1.000601,19692.00,1.0,26290.0,478_0_1,-12.850165
2,2.0,478,0,599128.74,1.0,1.000584,4359198.25,NaN,NaN,0.999918,4488.22,1.000636,34955.12,1.0,26290.0,478_0_2,-0.439882
3,3.0,478,0,2872317.54,-1.0,0.999802,27129551.64,NaN,NaN,0.999705,16082.04,1.000189,10314.00,1.0,26290.0,478_0_3,7.259846
4,4.0,478,0,740059.14,-1.0,0.999886,8880890.78,NaN,NaN,0.999720,19012.35,1.000107,7245.60,1.0,26290.0,478_0_4,4.780292


In [54]:
df.rename(index = str, columns = {'revealed_target': 'target'}, inplace = True)

In [55]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,target
0,0.0,478,0,3753451.43,-1.0,0.999875,11548975.43,NaN,NaN,0.999875,22940.00,1.000050,9177.60,1.0,26290.0,478_0_0,-2.310276
1,1.0,478,0,985977.11,-1.0,1.000245,3850033.97,NaN,NaN,0.999940,1967.90,1.000601,19692.00,1.0,26290.0,478_0_1,-12.850165
2,2.0,478,0,599128.74,1.0,1.000584,4359198.25,NaN,NaN,0.999918,4488.22,1.000636,34955.12,1.0,26290.0,478_0_2,-0.439882
3,3.0,478,0,2872317.54,-1.0,0.999802,27129551.64,NaN,NaN,0.999705,16082.04,1.000189,10314.00,1.0,26290.0,478_0_3,7.259846
4,4.0,478,0,740059.14,-1.0,0.999886,8880890.78,NaN,NaN,0.999720,19012.35,1.000107,7245.60,1.0,26290.0,478_0_4,4.780292


In [60]:
X_test, y_test = generate_features(df, True)

/var/folders/8z/rmv13nvj523fxpvqhcn4k7dm0000gn/T/ipykernel_71144/3052505903.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.replace([np.inf, -np.inf], 0, inplace = True)


In [61]:
dtest = xgb.DMatrix(data = X_test, label = y_test)

y_pred = regressor.predict(dtest)
mae_test = mean_absolute_error(y_test, y_pred)

print("Mean Absolute Error: {}".format(mae_test))

Mean Absolute Error: 5.492850110322133


In [65]:
y_pred.shape

(32972,)